In [ ]:
import requests
import json
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)

In [ ]:
# fetch api key from file

with open('secret/key.txt', 'r') as file:
    api_key = file.read()
#api_key

In [ ]:
#keywords = ["swollen ankles", "horny goat weed", "scalp psoriasis"]

url = "https://google.serper.dev/search"
location = "Calgary,Alberta,Canada"
language = "en"
device = "desktop"

In [ ]:
# peforms a serch on one kw and returns the Xth page of relults

def search_kw(kw, page=1, url="https://google.serper.dev/search"):
    payload = json.dumps({
      "q": kw, # keyword to search for
      "gl": "us", # location
      "hl": "en", # language
      "autocorrect": False,
      "page": page
    })

    headers = {
      "X-API-KEY": api_key,
      "Content-Type": "application/json"
    }

    response = requests.request("POST", url=url, headers=headers, data=payload)
    return(response)

In [ ]:
# runs search_ky over multiple pages and returns them in a list

def get_pages(page_count, kw, url="https://google.serper.dev/search"):
    pages = list()
    for page in range(1, page_count+1):
        pages.append(search_kw(kw, page, url))
    return pages

In [ ]:
# extract kw, page, rank, and link from 1 page of response info and return df

def extract_response(response):
    df = pd.DataFrame(columns = ['kw', 'page', 'rank','link'])
    q = response.json()['searchParameters']['q']
    page = response.json()['searchParameters']['page']
    for rank, result in enumerate(response.json()["organic"], 1):
        link = result['link']
        df.loc[len(df)] = [q, page, rank, link]
    return(df)

In [ ]:
# takes a list of pages of response, iterates extract_response(), extracts and renumbers responses, returns df of ky, rank, link

def collate_pages(pages, max_rank=None):
    df = pd.DataFrame()
    for response in pages:
        data = extract_response(response)
        df = pd.concat([df, data], ignore_index=True)
    df = df.reset_index(drop=True)
    df["rank"] = df.index + 1
    if max_rank is not None:
        df = df[df['rank'] <= max_rank]
    df.drop(["page"], axis=1, inplace=True)
    return df

In [ ]:
# accepty a df containing "Keywords" and iterates collate_pages() returns df of kw

def get_links_for_kws(df_in):
    df_out = pd.DataFrame()
    keywords = df_in["Keyword"]
    for kw in keywords:
        kw_pages = get_pages(3, kw) # get 3 pages (<= 30 records) for each kw
        df = collate_pages(kw_pages, 20) # colate pages, limit to top 20 records.
        df_out = pd.concat([df_out, df], ignore_index=True)
    return(df_out)

In [ ]:
# load kw
df_in = pd.read_csv('data/kw_1_000.csv')
#df_in = df_in.head() # FOR TESTING PURPOSES ONLY BECAUSE I AM LIMITED BY MY QUERY QUOTA OF 1440
#df_in.iloc[400:500]

In [ ]:
df_400 = df_in.iloc[0:400]
df_400_800 = df_in.iloc[400:800]
df_800_1000 = df_in.iloc[800:]

In [ ]:
#df_400_800.tail()

,Keyword,SERP features,Volume,URL
795,tattoo numbing cream,"People also ask, Shopping results, Thumbnail",51000,https://www.amazon.com/tattoo-numbing-cream/s?...
796,adjustable facemask,Sitelinks,12000,https://www.amazon.com/Cloth-Washable-Reusable...
797,living room office ideas,"Image pack, People also ask, Shopping results,...",1200,https://www.thespruce.com/desk-in-living-room-...
798,orchid cactus,"People also ask, Knowledge panel, Shopping res...",11000,https://www.thespruce.com/epiphyllum-5212346
799,casodex,"Thumbnail, People also ask, Knowledge panel, S...",10000,https://www.webmd.com/drugs/2/drug-4510/casode...


In [ ]:
# REMEMBER YOU ARE LIMITED BY A DAILY QUERY QUOTA OF 1440

In [20]:
#df = get_links_for_kws(df_400)
df = get_links_for_kws(df_400_800)
#df = get_links_for_kws(df_800_1000)

In [ ]:
#df.to_csv('data/first_400.csv')
df.to_csv('data/kw_400_800.csv')

In [ ]:
df

,kw,rank,link
0,side table lamps,1,https://www.target.com/c/table-lamps-lighting-...
1,side table lamps,2,https://www.amazon.com/Side-Table-Lamps/s?k=Si...
2,side table lamps,3,https://www.wayfair.com/lighting/sb1/bedside-t...
3,side table lamps,4,https://www.ikea.com/us/en/cat/table-lamps-10732/
4,side table lamps,5,https://www.crateandbarrel.com/lighting/table-...
...,...,...,...
7985,raisins nutrition,16,https://www.checkyourfood.com/ingredients/ingr...
7986,raisins nutrition,17,https://healthyeating.sfgate.com/health-benefi...
7987,raisins nutrition,18,https://www.wellandgood.com/benefits-of-raisins/
7988,raisins nutrition,19,https://ratinkhosh.com/disadvantages-of-eating...


In [ ]:
# prints the response for 1 keyword

def print_response(response):
    print (f"Results for keyword: \"{response.json()['searchParameters']['q']}\"")
    for rank, result in enumerate(response.json()["organic"], 1):
        print(f"Rank: {rank}, Title: {result['title']}, Link: {result['link']}")